In [60]:
import re
import camelot
import os
import csv

In [61]:
file_path = 'dats24-elektriciteit-2023-01.pdf'

In [62]:
data = dict()

In [63]:
tables = camelot.read_pdf(file_path, pages="all", flavor="stream")

Overwriting cache for 0 5634
Overwriting cache for 0 5634
Overwriting cache for 0 5634
Overwriting cache for 0 5634
Overwriting cache for 0 5634
C:\Users\jaakd\PycharmProjects\dep1-24-25-g02\venv\Lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (11.36, 448.36, 580.2311199999996, 708.7219999999998)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


In [64]:
for i, table in enumerate(tables):
    df = table.df
    
    if df.isin(["VASTE VERGOEDING (€/jaar)"]).any().any():
        print(f"'VASTE VERGOEDING (€/jaar)' found in Table {i+1}")
        fixed_fee_row = df.index[df.isin(["VASTE VERGOEDING (€/jaar)"]).any(axis=1)][0]
        data['AdministrativeCosts'] = df.loc[fixed_fee_row][1]
    
    if df.isin(["Afname1 (c€/kWh)"]).any().any():
        print(f"'Afname1 (c€/kWh)' found in Table {i+1}")
        variable_row = df.index[df.isin(["Afname1 (c€/kWh)"]).any(axis=1)][0]
        data['SingleMeterFixed'] = df.loc[variable_row][1]
        data['DualMeterDayFixed'] = df.loc[variable_row][2]
        data['DualMeterNightFixed'] = df.loc[variable_row][4]
        data['ExclusiveNightMeterFixed'] = df.loc[variable_row][5]

    if df.isin(["Teruglevering2 (c€/kWh)"]).any().any():
        print(f"'Teruglevering2 (c€/kWh)' found in Table {i+1}")
        injection_row = df.index[df.isin(["Teruglevering2 (c€/kWh)"]).any(axis=1)][0]
        data['SingleMeterInjectionMeterFactor'] = df.loc[injection_row][1]
        data['DualMeterDayInjectionMeterFactor'] = df.loc[injection_row][3]
        data['DualMeterNightInjectionMeterFactor'] = df.loc[injection_row][3]

    if df.isin(["GSC (c€/kWh)"]).any().any():
        print(f"'GSC (c€/kWh)' found in Table {i+1}")
        gsc_row = df.index[df.isin(["GSC (c€/kWh)"]).any(axis=1)][0]
        data['GreenElectricity'] = df.loc[gsc_row][1]

    if df.isin(["WKC (c€/kWh)"]).any().any():
        print(f"'WKC (c€/kWh)' found in Table {i+1}")
        wkc_row = df.index[df.isin(["WKC (c€/kWh)"]).any(axis=1)][0]
        data['WKK'] = df.loc[wkc_row][1]
    
    for idx in range(len(df)):
        row_values = df.iloc[idx].tolist()
        if any('GSC' in str(val) for val in row_values if val is not None):
            print(f"'GSC' found in Table {i+1} at row {idx}")
            data['GreenElectricity'] = df.iloc[idx][1]
            break

    print(data)
    break


'VASTE VERGOEDING (€/jaar)' found in Table 1
'Afname1 (c€/kWh)' found in Table 1
'Teruglevering2 (c€/kWh)' found in Table 1
'WKC (c€/kWh)' found in Table 1
'GSC' found in Table 1 at row 13
{'AdministrativeCosts': '38,50', 'SingleMeterFixed': '32,48', 'DualMeterDayFixed': '38,49', 'DualMeterNightFixed': '29,50', 'ExclusiveNightMeterFixed': '29,50', 'SingleMeterInjectionMeterFactor': '21,16', 'DualMeterDayInjectionMeterFactor': '21,16', 'DualMeterNightInjectionMeterFactor': '21,16', 'WKK': '0,357', 'GreenElectricity': '2,210'}


In [65]:
for i, table in enumerate(tables):
    df = table.df
    
    if df.isin(["TARRIEFFORMULE"]).any().any():
        print(f"Found tariff formula table (Table {i+1})")
        
        for idx in range(len(df)):
            row_values = df.iloc[idx].tolist()
            
            if "Enkelvoudige meter" in row_values:
                formula = df.iloc[idx][2]  # AFNAME column
                if "BE_spotRLP" in formula:
                    factor = formula.split('x')[1].split('+')[0].strip()
                    balancing = formula.split('+')[1].split(')')[0].strip()
                    data['SingleMeterVariableMeterFactor'] = factor
                    data['SingleMeterVariableBalancingCost'] = balancing
            
            if "Tweevoudige meter" in row_values and "Dag" in row_values:
                formula = df.iloc[idx][2]
                if "BE_spotRLP" in formula:
                    data['DualMeterDayVariableMeterFactor'] = formula.split('x')[1].split('+')[0].strip()
                    data['DualMeterDayVariableBalancingCost'] = formula.split('+')[1].split(')')[0].strip()
            
            if "Nacht" in row_values:
                formula = df.iloc[idx][2]
                if "BE_spotRLP" in formula:
                    data['DualMeterNightVariableMeterFactor'] = formula.split('x')[1].split('+')[0].strip()
                    data['DualMeterNightVariableBalancingCost'] = formula.split('+')[1].split(')')[0].strip()
            
            if "Uitsluitend nachtmeter" in row_values:
                formula = df.iloc[idx][2]
                if "BE_spotRLP" in formula:
                    data['ExclusiveNightMeterVariableMeterFactor'] = formula.split('x')[1].split('+')[0].strip()
                    data['ExclusiveNightMeterVariableBalancingCost'] = formula.split('+')[1].split(')')[0].strip()
            
            if len(row_values) > 3 and isinstance(row_values[3], str) and "BE_spotSPP" in row_values[3]:
                injection = row_values[3]
                injection_factor = injection.split('x')[1].split('-')[0].strip()
                injection_balancing = injection.split('-')[1].strip().split(')')[0].strip()
                data['SingleMeterInjectionMeterFactor'] = injection_factor
                data['SingleMeterInjectionBalancingCost'] = injection_balancing
                data['DualMeterDayInjectionMeterFactor'] = injection_factor
                data['DualMeterDayInjectionBalancingCost'] = injection_balancing
                data['DualMeterNightInjectionMeterFactor'] = injection_factor
                data['DualMeterNightInjectionBalancingCost'] = injection_balancing
        
        print(data)
        break


Found tariff formula table (Table 2)
{'AdministrativeCosts': '38,50', 'SingleMeterFixed': '32,48', 'DualMeterDayFixed': '38,49', 'DualMeterNightFixed': '29,50', 'ExclusiveNightMeterFixed': '29,50', 'SingleMeterInjectionMeterFactor': '0,073', 'DualMeterDayInjectionMeterFactor': '0,073', 'DualMeterNightInjectionMeterFactor': '0,073', 'WKK': '0,357', 'GreenElectricity': '2,210', 'SingleMeterVariableMeterFactor': '0,1084', 'SingleMeterVariableBalancingCost': '0,697', 'DualMeterDayVariableMeterFactor': '0,1289', 'DualMeterDayVariableBalancingCost': '0,697', 'SingleMeterInjectionBalancingCost': '0,38', 'DualMeterDayInjectionBalancingCost': '0,38', 'DualMeterNightInjectionBalancingCost': '0,38', 'DualMeterNightVariableMeterFactor': '0,0982', 'DualMeterNightVariableBalancingCost': '0,697', 'ExclusiveNightMeterVariableMeterFactor': '0,0982', 'ExclusiveNightMeterVariableBalancingCost': '0,697'}


In [66]:
year, month = re.search(r'(\d{4})-(\d{2})', file_path).groups()
date_key = f"{year}{month}01"  # Format as yyyymmdd with day as 01

In [67]:
csv_file = "dats24.csv"

In [68]:
file_exists = os.path.isfile(csv_file)

In [69]:
data = {key: value.replace(',', '.') if isinstance(value, str) else value for key, value in data.items()}

In [70]:
with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    
    # Write header only if file doesn't exist
    if not file_exists:
        header = ["DateKey", "ContractKey", "SingleMeterFixed", "DualMeterDayFixed", 
                  "DualMeterNightFixed", "ExclusiveNightMeterFixed", "SingleMeterVariableMeterFactor", 
                  "SingleMeterVariableBalancingCost", "DualMeterDayVariableMeterFactor", 
                  "DualMeterDayVariableBalancingCost", "DualMeterNightVariableMeterFactor", 
                  "DualMeterNightVariableBalancingCost", "ExclusiveNightMeterVariableMeterFactor", 
                  "ExclusiveNightMeterVariableBalancingCost", "DynamicMeterCost", "DynamicBalancingCost", 
                  "SingleMeterInjectionMeterFactor", "SingleMeterInjectionBalancingCost", 
                  "DualMeterDayInjectionMeterFactor", "DualMeterDayInjectionBalancingCost", 
                  "DualMeterNightInjectionMeterFactor", "DualMeterNightInjectionBalancingCost", 
                  "AdministrativeCosts", "GreenElectricity", "WKK"]
        writer.writerow(header)
    
    # Prepare data row in the specified order
    row_data = [
        date_key,
        "DATS24_ELEKTRICITEIT",
        data['SingleMeterFixed'],
        data['DualMeterDayFixed'],
        data['DualMeterNightFixed'],
        data['ExclusiveNightMeterFixed'],
        data['SingleMeterVariableMeterFactor'],
        data['SingleMeterVariableBalancingCost'],
        data['DualMeterDayVariableMeterFactor'],
        data['DualMeterDayVariableBalancingCost'],
        data['DualMeterNightVariableMeterFactor'],
        data['DualMeterNightVariableBalancingCost'],
        data['ExclusiveNightMeterVariableMeterFactor'],
        data['ExclusiveNightMeterVariableBalancingCost'],
        data.get('DynamicMeterCost', ''),
        data.get('DynamicBalancingCost', ''),
        data['SingleMeterInjectionMeterFactor'],
        data['SingleMeterInjectionBalancingCost'],
        data['DualMeterDayInjectionMeterFactor'],
        data['DualMeterDayInjectionBalancingCost'],
        data['DualMeterNightInjectionMeterFactor'],
        data['DualMeterNightInjectionBalancingCost'],
        data['AdministrativeCosts'],
        data['GreenElectricity'],
        data['WKK']
    ]
    
    # Write data row
    writer.writerow(row_data)

print(f"Data for {date_key} successfully written to {csv_file}")

Data for 20230101 successfully written to dats24.csv
